记录日志

In [ ]:
#Runner对象的配置关键字为：log_processor,以下为默认配置
log_processor = dict(window_size=10, by_epoch=True, custom_cfg=None, num_digits=4)


In [ ]:
#默认配置的输出
#08/21 02:58:41 - mmengine - INFO - Epoch(train) [1][10/25]  lr: 1.0000e-02  eta: 0:00:00  time: 0.0019  data_time: 0.0004  loss1: 0.8381  loss2: 0.9007  loss: 1.7388
#08/21 02:58:41 - mmengine - INFO - Epoch(train) [1][20/25]  lr: 1.0000e-02  eta: 0:00:00  time: 0.0029  data_time: 0.0010  loss1: 0.1978  loss2: 0.4312  loss: 0.6290

window_size表示统计范围 \
    1. 为数字表示迭代次数（一个训练样本的一个iter） \
    2. global为全局，即动态地从开始到当前 \
    3. epoch，即一个epoch范围内 \
by_epoch表示统计按照epoch，为False表示按照迭代iter \
    1. 这个参数必须和train_cfg保持一致 \
num_digits表示输出数值的有效位数 \
custom_cfg表示自定义字段 \
    1. 如果要修改某个字段比如loss1而给所有字段的统计范围； \
    2. 为某个字段如loss1添加一个统计范围；\

In [ ]:
from mmengine.runner import Runner
runner = Runner(
    model=ToyModel(),
    work_dir='tmp_dir',
    train_dataloader=train_dataloader,
    train_cfg=dict(by_epoch=True, max_epochs=1),
    optim_wrapper=dict(optimizer=dict(type='SGD', lr=0.01)),
    log_processor=dict(  # 配置日志处理器
        custom_cfg=[
            dict(data_src='loss1',  # 原日志名：loss1
                 method_name='mean',  # 统计方法：均值统计
                 window_size='global')])  # 统计窗口：全局
)
runner.train()

#data_src字段表示原日志统计的字段名
#method_name字段表示统计方法，可选项除了均值mean还有最大值max，最小值min，当前current（这种不需要指定统计范围window_size参数）

runner = Runner(
    model=ToyModel(),
    work_dir='tmp_dir',
    train_dataloader=train_dataloader,
    train_cfg=dict(by_epoch=True, max_epochs=1),
    optim_wrapper=dict(optimizer=dict(type='SGD', lr=0.01)),
    log_processor=dict(custom_cfg=[
        # 统计 loss1 的局部最大值，统计窗口为 10，并在日志中重命名为 loss1_local_max
        dict(data_src='loss1',
             log_name='loss1_local_max',
             window_size=10,
             method_name='max'),
        # 统计 loss1 的全局最大值，并在日志中重命名为 loss1_local_max
        dict(
            data_src='loss1',
            log_name='loss1_global_max',
            method_name='max',
            window_size='global')
    ]))
runner.train()

自定义统计内容 \
即除了损失、学习率等可以添加其他信息，比如特征图与标签计算损失

In [ ]:
#下面给出一个示例

#整个步骤可以梳理为：
#1. 使用消息枢纽的get_current_instance接口获取执行器当前的消息枢纽 # TODO 消息枢纽https://mmengine.readthedocs.io/zh_CN/latest/api/generated/mmengine.logging.MessageHub.html#mmengine.logging.MessageHub
#2. 使用update_scalar接口添加统计内容，第一个参数为参数名，以train/,val/,test/开头,后面才是实际名
#3. 在执行器的log_processor字段中设置custom_cfg




from mmengine.logging import MessageHub


class ToyModel(BaseModel):

    def __init__(self) -> None:
        super().__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, img, label, mode):
        feat = self.linear(img)
        loss_tmp = (feat - label).abs()
        loss = loss_tmp.pow(2)

        message_hub = MessageHub.get_current_instance()
        # 在日志中额外统计 `loss_tmp`
        message_hub.update_scalar('train/loss_tmp', loss_tmp.sum())
        return dict(loss=loss)


runner = Runner(
    model=ToyModel(),
    work_dir='tmp_dir',
    train_dataloader=train_dataloader,
    train_cfg=dict(by_epoch=True, max_epochs=1),
    optim_wrapper=dict(optimizer=dict(type='SGD', lr=0.01)),
    log_processor=dict(
        custom_cfg=[
        # 统计 loss_tmp 的局部均值
            dict(
                data_src='loss_tmp',
                window_size=10,
                method_name='mean')
        ]
    )
)
runner.train()

输出调试日志 \
Runner对象的log_level字段，即输出日志等级，比如debug，默认为info